<img align="left" src="https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/CC_BY.png"><br />

Created by [Nathan Kelber](http://nkelber.com) and Ted Lawless for [JSTOR Labs](https://labs.jstor.org/) under [Creative Commons CC BY License](https://creativecommons.org/licenses/by/4.0/)<br />
For questions/comments/improvements, email nathan.kelber@ithaka.org.<br />
___

# Latent Dirichlet Allocation (LDA) Topic Modeling

**Description:**
This [notebook](https://docs.constellate.org/key-terms/#jupyter-notebook) demonstrates how to do topic modeling. The following processes are described:

* Using the `tdm_client` to retrieve a dataset
* Filtering based on a pre-processed ID list
* Filtering based on a [stop words list](https://docs.constellate.org/key-terms/#stop-words)
* Cleaning the tokens in the dataset
* Creating a [gensim dictionary](https://docs.constellate.org/key-terms/#gensim-dictionary)
* Creating a [gensim](https://docs.constellate.org/key-terms/#gensim) [bag of words](https://docs.constellate.org/key-terms/#bag-of-words) [corpus](https://docs.constellate.org/key-terms/#corpus)
* Computing a topic list using [gensim](https://docs.constellate.org/key-terms/#gensim)
* Visualizing the topic list with `pyldavis`

**Use Case:** For Researchers (Mostly code without explanation, not ideal for learners)

**Difficulty:** Intermediate

**Completion time:** 30 minutes

**Knowledge Required:** 
* Python Basics Series ([Start Python Basics I](./python-basics-1.ipynb))

**Knowledge Recommended:**
* [Exploring Metadata](./metadata.ipynb)
* [Working with Dataset Files](./working-with-dataset-files.ipynb)
* [Pandas I](./pandas-1.ipynb)
* [Creating a Stopwords List](./creating-stopwords-list.ipynb)
* A familiarity with [gensim](https://docs.constellate.org/key-terms/#gensim) is helpful but not required.

**Data Format:** [JSON Lines (.jsonl)](https://docs.constellate.org/key-terms/#jsonl)

**Libraries Used:**
* `pandas` to load a preprocessing list
* `csv` to load a custom stopwords list
* [gensim](https://docs.constellate.org/key-terms/#gensim) to accomplish the topic modeling
* [NLTK](https://docs.constellate.org/key-terms/#nltk) to create a stopwords list (if no list is supplied)
* `pyldavis` to visualize our topic model

**Research Pipeline**
1. Build a dataset
2. Create a "Pre-Processing CSV" with [Exploring Metadata](./exploring-metadata.ipynb) (Optional)
3. Create a "Custom Stopwords List" with [Creating a Stopwords List](./creating-stopwords-list.ipynb) (Optional)
4. Complete the Topic Modeling analysis with this notebook
____

## What is Topic Modeling?

**Topic modeling** is a **machine learning** technique that attempts to discover groupings of words (called topics) that commonly occur together in a body of texts. The body of texts could be anything from journal articles to newspaper articles to tweets.

**Topic modeling** is an unsupervised, clustering technique for text. We give the machine a series of texts that it then attempts to cluster the texts into a given number of topics. There is also a *supervised*, clustering technique called **Topic Classification**, where we supply the machine with examples of pre-labeled topics and then see if the machine can identify them given the examples.

**Topic modeling** is usually considered an exploratory technique; it helps us discover new patterns within a set of texts. **Topic Classification**, using labeled data, is intended to be a predictive technique; we want it to find more things like the examples we give it.

## Import your dataset

We'll use the tdm_client library to automatically retrieve the dataset in the JSON file format. 

Enter a [dataset ID](https://docs.constellate.org/key-terms/#dataset-ID) in the next code cell.

If you don't have a dataset ID, you can:
* Use the sample dataset ID already in the code cell
* [Create a new dataset](https://constellate.org/builder)
* [Use a dataset ID from other pre-built sample datasets](https://constellate.org/dataset/dashboard)

In [1]:
# Creating a variable `dataset_id` to hold our dataset ID
# Not from default dataset.

# The default dataset is Independent Voices
# Independent Voices is an open access digital collection of alternative press newspapers, magazines and journals,
# drawn from the special collections of participating libraries. These periodicals were produced by feminists, 
# dissident GIs, campus radicals, Native Americans, anti-war activists, Black Power advocates, Hispanics, 
# LGBT activists, the extreme right-wing press and alternative literary magazines 
# during the latter half of the 20th century.

dataset_id = "9836c854-eb7c-612c-59ee-5f2dc57f3de9"

Next, import the `tdm_client`, passing the `dataset_id` as an argument using the `get_dataset` method.

In [2]:
# Importing your dataset with a dataset ID
import tdm_client
# Pull in the dataset that matches `dataset_id`
# in the form of a gzipped JSON lines file.
dataset_file = tdm_client.get_dataset(dataset_id)

INFO:root:Downloading 9836c854-eb7c-612c-59ee-5f2dc57f3de9 metadata to /home/jovyan/data/9836c854-eb7c-612c-59ee-5f2dc57f3de9-sampled-jsonl.jsonl.gz


100% |########################################################################|


## Apply Pre-Processing Filters (if available)
If you completed pre-processing with the "Exploring Metadata and Pre-processing" notebook, you can use your CSV file of dataset IDs to automatically filter the dataset. Your pre-processed CSV file  must be in the root folder.

In [3]:
# Import a pre-processed CSV file of filtered dataset IDs.
# If you do not have a pre-processed CSV file, the analysis
# will run on the full dataset and may take longer to complete.
import pandas as pd
import os

pre_processed_file_name = f'data/pre-processed_{dataset_id}.csv'

if os.path.exists(pre_processed_file_name):
    df = pd.read_csv(pre_processed_file_name)
    filtered_id_list = df["id"].tolist()
    use_filtered_list = True
    print('Pre-Processed CSV found. Successfully read in ' + str(len(df)) + ' documents.')
else: 
    use_filtered_list = False
    print('No pre-processed CSV file found. Full dataset will be used.')

INFO:numexpr.utils:NumExpr defaulting to 4 threads.
Pre-Processed CSV found. Successfully read in 1205 documents.


## Load Stopwords List

If you have created a stopword list in the stopwords notebook, we will import it here. (You can always modify the CSV file to add or subtract words then reload the list.) Otherwise, we'll load the NLTK [stopwords](https://docs.constellate.org/key-terms/#stop-words) list automatically.

In [4]:
# Load a custom data/stop_words.csv if available
# Otherwise, load the nltk stopwords list in English

# The filename of the custom data/stop_words.csv file
stopwords_list_filename = 'data/stop_words.csv'

if os.path.exists(stopwords_list_filename):
    import csv
    with open(stopwords_list_filename, 'r') as f:
        stop_words = list(csv.reader(f))[0]
    print('Custom stopwords list loaded from CSV')
else:
    # Load the NLTK stopwords list
    from nltk.corpus import stopwords
    stop_words = stopwords.words('english')
    print('NLTK stopwords list loaded')

NLTK stopwords list loaded


In [5]:
list(stop_words)

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

## Define a Function to Process Tokens
Next, we create a short function to clean up our tokens.

In [7]:
import string

def process_token(token):
    token = token.lower()
    token = token.strip(string.punctuation)
    if token in stop_words:
        return
    if len(token) < 4:
        return
    if not(token.isalpha()):
        return
    return token

In [8]:
# Limit to n documents. Set to None to use all documents.

limit = 1500

n = 0
documents = []
for document in tdm_client.dataset_reader(dataset_file):
    processed_document = []
    document_id = document["id"]
    if use_filtered_list is True:
        # Skip documents not in our filtered_id_list
        if document_id not in filtered_id_list:
            continue
    unigrams = document.get("unigramCount", {})
    for gram, count in unigrams.items():
        clean_gram = process_token(gram)
        if clean_gram is None:
            continue
        processed_document += [clean_gram] * count # Add the unigram as many times as it was counted
    if len(processed_document) > 0:
        documents.append(processed_document)
    n += 1
    if (limit is not None) and (n >= limit):
       break

Build a gensim dictionary corpus and then train the model. More information about parameters can be found at the [Gensim LDA Model page](https://radimrehurek.com/gensim/models/ldamodel.html).

In [9]:
import gensim
dictionary = gensim.corpora.Dictionary(documents)

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(111709 unique tokens: ['absolute', 'abstract', 'achieve', 'acts', 'addition']...) from 1205 documents (total 3806880 corpus positions)


In [10]:
doc_count = len(documents)
num_topics = 7 # Change the number of topics
passes = 5 # The number of passes the model runs
# Remove terms that appear in less than 10 documents and terms that occur in more than 50% of documents.
dictionary.filter_extremes(no_below=10, no_above=0.5)

INFO:gensim.corpora.dictionary:discarding 100820 tokens: [('abstract', 662), ('along', 674), ('analysis', 761), ('applied', 769), ('blankets', 2), ('cantilevered', 6), ('case', 1043), ('cases', 711), ('closed', 658), ('concur', 7)]...
INFO:gensim.corpora.dictionary:keeping 10889 tokens which were in no less than 10 and no more than 602 (=50.0%) documents
INFO:gensim.corpora.dictionary:resulting dictionary: Dictionary(10889 unique tokens: ['absolute', 'achieve', 'acts', 'addition', 'additional']...)


In [11]:
bow_corpus = [dictionary.doc2bow(doc) for doc in documents]

In [12]:
# Train the LDA model
model = gensim.models.LdaModel(
    corpus=bow_corpus,
    id2word=dictionary,
    num_topics=num_topics,
    passes=passes
)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.14285714285714285
INFO:gensim.models.ldamodel:using symmetric eta at 0.14285714285714285
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online (multi-pass) LDA training, 7 topics, 5 passes over the supplied corpus of 1205 documents, updating model once every 1205 documents, evaluating perplexity every 1205 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamodel:-9.825 per-word bound, 906.7 perplexity estimate based on a held-out corpus of 1205 documents with 2664300 words
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #1205/1205
INFO:gensim.models.ldamodel:topic #6 (0.143): 0.006*"magnetic" + 0.004*"proof" + 0.004*"vector" + 0.003*"control" + 0.003*"current" + 0.003*"lemma" + 0.003*"group" + 0.003*"math" + 0.003*"flow" + 0.002*"surface"
INFO:gensim.models.ldamodel:topic #0 (0.143): 0.004*"proof" + 0.004*"group" + 0.003*"magnetic"

## Display a List of Topics
Print the most significant terms, as determined by the model, for each topic.

In [13]:
for topic_num in range(0, num_topics):
    word_ids = model.get_topic_terms(topic_num)
    words = []
    for wid, weight in word_ids:
        word = dictionary.id2token[wid]
        words.append(word)
    print("Topic {}".format(str(topic_num).ljust(5)), " ".join(words))

Topic 0     continuum data choice response reinforcement behavior continua table experiment rate
Topic 1     solution velocity vortex numerical flow optimization boundary problems algorithm vector
Topic 2     potential gauge magnetic quantum vector energy integral phys surface force
Topic 3     flow periodic shape attractor maps vector bundle proof chaotic stable
Topic 4     control design robot controller position motion parameters durch data pressure
Topic 5     group proof lemma compact math groups exists proposition sequence maps
Topic 6     magnetic current coil surface coils data flux frequency density axis


## Visualize the Topic Distances on a Flat Plane

Visualize the model using [`pyLDAvis`](https://pyldavis.readthedocs.io/en/latest/). This visualization can take a while to generate depending on the size of your dataset.

In [14]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model, bow_corpus, dictionary)

/srv/conda/envs/notebook/lib/python3.7/site-packages/pyLDAvis/_prepare.py:248: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.214490 -0.033138       1        1  26.089193
6     -0.088042  0.049312       2        1  20.103553
2      0.020221  0.110372       3        1  16.044241
4     -0.136790 -0.046477       4        1  13.760136
0     -0.039144 -0.088975       5        1   9.602508
1     -0.050119  0.010270       6        1   9.511097
3      0.079385 -0.001363       7        1   4.889273, topic_info=          Term          Freq         Total Category  logprob  loglift
1058   control   7619.000000   7619.000000  Default  30.0000  30.0000
494      proof  10345.000000  10345.000000  Default  29.0000  29.0000
129   magnetic  11851.000000  11851.000000  Default  28.0000  28.0000
770      lemma   8534.000000   8534.000000  Default  27.0000  27.0000
394      group  12796.000000  12796.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
297    compact    383.528973   7760.270417   Topic7  -5.8279   0.0108
2524    period    273.514504   1644.392922   Topic7  -6.1660   1.2244
530       sets    312.412929   4918.411815   Topic7  -6.0330   0.2617
529   sequence    314.747615   5307.478281   Topic7  -6.0256   0.1930
770      lemma    321.285567   8534.322842   Topic7  -6.0050  -0.2614

[573 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
2095      1  0.852574  abelian
2095      2  0.010781  abelian
2095      3  0.092984  abelian
2095      4  0.008984  abelian
2095      5  0.014374  abelian
...     ...       ...      ...
4806      3  0.009586     zwar
4806      4  0.939394     zwar
4806      5  0.009586     zwar
4806      6  0.009586     zwar
4806      7  0.009586     zwar

[3370 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 7, 3, 5, 1, 2, 4])